In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
master_path = 'C:/Projects/DataCleanup/Data/'
master_file = 'Data Cleanup Tracking Sheet.xlsx'
master_sheet = 'Full Tracking List'
#try:
file_df = pd.read_excel(master_path + master_file, sheet_name=master_sheet)
#file_df = pd.read_csv(master_path + master_file, low_memory=False, index_col='ProfileID_M')
#except:
#    OverflowError as of:
file_df

,FirstName_M,LastName_M,Phone_M,email_M,Class_M,Specialty_M,Address_M,City_M,State_M,Zip_M,...,GroupsUpdated,Removed_AM,AM_By:,MobilizeUpdated,Removed_Mob,MobilizeBy:,BackendUpdated,BE_By:,DNR,Completed
0,Fatimah,Abdullah,000-000-0000,ameenah007@yahoo.com,Certified Nursing Assistant,Geriatrics,30760 eden allen road,Eden,MD,21822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,?,NaT
1,Julie,Szabo,413-770-7671,szabo.julie@yahoo.com,Licensed Practical Nurse,Geriatrics; Clinic,12 von nida ave,Pittsfield,MA,1201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,David,Barsamian,000-000-0000,noemail@favoritestaffing.com,Registered Nurse,Medical/Surgical; Telemetry,380 east center st,Manchester,CT,6040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,Tracy,Seelbaugh,724-372-4466; 724-712-4688,tlseelbaugh@gmail.com,Registered Nurse; Certified Nursing Assistant,Geriatrics,40 Enclave Dr,Grove City,PA,16127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,Karen,Wlostoski,413-773-7298,sasha65@comcast.net,Registered Nurse,Medical/Surgical; Geriatrics; Telemetry; In...,PO Box 532,Greenfield,MA,1302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1048571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1048572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1048573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [3]:
# Load list of deletes for not interested responses

In [4]:
not_int_df = pd.read_excel(master_path + "am_Nos.xlsx")
not_int_df


,Profile ID in Mobilize,Name,Phone,email,Response,Ok to delete
0,26777.0,Joseph (Michael) Battig,5.417478e+09,michaelbattig@gmail.com,Not Interested,Yes
1,33846.0,Reginald Hills,9.404010e+09,thills2023@gmail.com,Not Interested,Yes
2,39383.0,Tori Marsh,3.605607e+09,torisfarm@aol.com,Not Interested,Yes
3,39715.0,Giovanna Barone,9.084723e+09,joann623@yahoo.com,Not Interested,Yes
4,41670.0,Robert Haberle,9.416620e+09,misouritraveller@aol.com,Not Interested,Yes
...,...,...,...,...,...,...
11152,969754.0,Khisha Sheppard,8.329102e+09,khisha_b@yahoo.com,Not Interested,Yes
11153,979670.0,Geraldine Jackson,9.125083e+09,dinawilliamsjackson1@gmail.com,Not Interested,Yes
11154,986961.0,George Fiakpoe,7.707151e+09,gfiakpoe@yahoo.com,Not Interested,Yes
11155,1029174.0,Sara Barber,7.195027e+09,saraebarb@gmail.com,Not Interested,Yes


<H1>Create New Tracking list without all the rows at the end

In [5]:
file_df.dropna(subset=['FirstName_M', 'LastName_M'], inplace=True)

In [6]:
not_int_df = not_int_df.assign(result=not_int_df['email'].isin(file_df['email_M']))

In [7]:
# Mark anyone in file_df as delete if in the above list
file_df = file_df.assign(result=file_df['email_M'].isin(not_int_df['email']))
file_df = file_df.assign(result=file_df['ProfileID_M'].isin(not_int_df['Profile ID in Mobilize']))

In [8]:
file_df['AM Delete'] = file_df['result']
mask = file_df['AM Delete'] == True
file_df['KeeporDelete'][mask] = "Delete"
file_df['DecisionBy:'][mask] = 'Julie H.'
file_df['Removed_AM'][mask] = 'Yes'
file_df['AM_By:'][mask] = 'Connor'
file_df['Removed_Mob'][mask] = 'Yes'
file_df['MobilizeBy:'][mask] = 'Julie H.'
file_df['Completed'][mask] = '10/09/2021'

file_df.drop(columns=['result'], inplace=True)

<ipython-input-8-4f606620863f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df['KeeporDelete'][mask] = "Delete"
<ipython-input-8-4f606620863f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df['DecisionBy:'][mask] = 'Julie H.'
<ipython-input-8-4f606620863f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df['Removed_AM'][mask] = 'Yes'
<ipython-input-8-4f606620863f>:6: SettingWithCopyWarning: 
A value is trying to 

In [9]:
file_df

,FirstName_M,LastName_M,Phone_M,email_M,Class_M,Specialty_M,Address_M,City_M,State_M,Zip_M,...,GroupsUpdated,Removed_AM,AM_By:,MobilizeUpdated,Removed_Mob,MobilizeBy:,BackendUpdated,BE_By:,DNR,Completed
0,Fatimah,Abdullah,000-000-0000,ameenah007@yahoo.com,Certified Nursing Assistant,Geriatrics,30760 eden allen road,Eden,MD,21822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,?,None
1,Julie,Szabo,413-770-7671,szabo.julie@yahoo.com,Licensed Practical Nurse,Geriatrics; Clinic,12 von nida ave,Pittsfield,MA,1201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,David,Barsamian,000-000-0000,noemail@favoritestaffing.com,Registered Nurse,Medical/Surgical; Telemetry,380 east center st,Manchester,CT,6040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,Tracy,Seelbaugh,724-372-4466; 724-712-4688,tlseelbaugh@gmail.com,Registered Nurse; Certified Nursing Assistant,Geriatrics,40 Enclave Dr,Grove City,PA,16127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,Karen,Wlostoski,413-773-7298,sasha65@comcast.net,Registered Nurse,Medical/Surgical; Geriatrics; Telemetry; In...,PO Box 532,Greenfield,MA,1302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369084,Sashauna,Brown,860-655-4979,Sashaunabrown30@gmail.com,Registered Nurse,Charge,NaN,NaN,NaN,6053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
369085,Chloe,Iverson,609-361-4626,chloeiverson73@yahoo.com,Patient Care Assistant/Tech,Intensive Care Unit,NaN,NaN,NaN,8050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
369086,Matthew,Hennessy,516-524-5514,hennessymatthew56@gmail.com,Paramedic,Emergency Management,911 Clydesdale Ct,Wilmington,NC,28411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
369087,Louis,Gaskins,937-286-1721,lgaskins2@hotmail.com,Warehouse Worker,Warehouse,NaN,NaN,NaN,40242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [10]:
not_int_df
not_int_df.to_excel(master_path + "Not Interested Not in Mobilize.xlsx")

<H1>Creating list of invalid contact info

In [11]:
mask_000 = file_df.loc[:,'Phone_M'].str.contains('000-000')
mask_111 = file_df.loc[:,'Phone_M'].str.contains('111-111')
mask_123 = file_df.loc[:,'Phone_M'].str.contains('123-123')
mask_999 = file_df.loc[:,'Phone_M'].str.contains('999-999')
mask_888 = file_df.loc[:,'Phone_M'].str.contains('888-888')
mask_noemail = file_df.loc[:,'email_M'].str.contains('noemail')
mask_fav = file_df.loc[:,'email_M'].str.contains('favoritestaf')
mask_invalid = file_df.loc[:,'email_M'].str.contains('invalid')
mask_dontcall = file_df.loc[:,'email_M'].str.contains('dontcall')
mask_0000 = file_df.loc[:,'email_M'].str.contains('0000')
mask_donot = file_df.loc[:,'email_M'].str.contains('donot')
mask_krucial = file_df.loc[:,'email_M'].str.contains('krucialstaf')
mask_1111 = file_df.loc[:,'email_M'].str.contains('1111')

In [12]:
bad_contact_df = file_df.loc[mask_000 | mask_111 | mask_123 | mask_999 | mask_888 | mask_noemail | mask_fav | mask_invalid | mask_dontcall |
           mask_0000 | mask_donot | mask_krucial | mask_1111, :]

In [13]:
bad_contact_df

,FirstName_M,LastName_M,Phone_M,email_M,Class_M,Specialty_M,Address_M,City_M,State_M,Zip_M,...,GroupsUpdated,Removed_AM,AM_By:,MobilizeUpdated,Removed_Mob,MobilizeBy:,BackendUpdated,BE_By:,DNR,Completed
0,Fatimah,Abdullah,000-000-0000,ameenah007@yahoo.com,Certified Nursing Assistant,Geriatrics,30760 eden allen road,Eden,MD,21822,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,?,None
2,David,Barsamian,000-000-0000,noemail@favoritestaffing.com,Registered Nurse,Medical/Surgical; Telemetry,380 east center st,Manchester,CT,6040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,Addie,Cobb,000-000-0000,favorite@favoritestaffing.com,Registered Nurse,Psychiatric; Medical/Surgical; Geriatrics,1414 Central Avenue,Alton,IL,62002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
15,Angela,Pagotto,000-000-0000,angelapagotto@sbcglobal.net,Registered Nurse; Licensed Practical Nurse; ...,Psychiatric; Medical/Surgical; Clinic; Tele...,27924 NE 53Rd Street,Camas,WA,98607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
18,Angie,Johnson,000-000-0000,angiejohnson00@yahoo.com,Registered Nurse,Medical/Surgical; Telemetry,12610 Driftwood Ct,Apple Valley,MN,55124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361047,Pablo,Pimeael,000-000-0000; 786-786-4442,NoEmail@krucialstaffing.com/Pablo87pimentel@ya...,Registered Nurse,Pediatrics; Intensive Care Unit,1990 Marseille Dr,Miami Beach,FL,33141-0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
361262,Yolanda,Smith,225-508-0434,yofasa30000@yahoo.com,Licensed Practical Nurse,Healthcare,P.O. Box 597 2226 South Albert Stre,Lutcher,LA,70071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
363048,Marva,Lambert,000-000-0000,donotcontact@donotcontact.com,EKG Technician,Emergency Room,20619 109 Ave,Queens,NY,11429,...,NaN,NaN,Julie H.,NaN,Yes,Julie H.,NaN,NaN,NaN,1633132800000000000
366664,Omega,Augustus,000-000-0000,omegaaugustus@gmail.com,Registered Nurse,Emergency Room,164 East Path Rise,West Henrietta,NY,14586-0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [14]:
bad_contact_df.to_excel(master_path + "bad_contact.xlsx")

In [15]:
file_df = file_df.assign(result=file_df['ProfileID_M'].isin(bad_contact_df['ProfileID_M']))
file_df['Bad contact info'] = file_df['result']
file_df.drop(columns=['result'], inplace=True)

In [16]:
# Make sure to not worry about terminated employees
term_df = pd.read_csv(master_path + "EmployeeListingReport.csv")

In [17]:
file_df.drop(file_df[file_df['AM Delete']=="TRUE"])

C:\Users\julie\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


KeyError: "['FirstName_M' 'LastName_M' 'Phone_M' 'email_M' 'Class_M' 'Specialty_M'\n 'Address_M' 'City_M' 'State_M' 'Zip_M' 'ProfileID_M' 'WorkdayID_M'\n 'DOB_M' 'Created_M' 'Modified_M' 'FirstName_AM' 'LastName_AM' 'Email_AM'\n 'Phone_AM' 'Address_AM' 'Address2_AM' 'City_AM' 'State_AM' 'Zip_AM'\n 'Gender_AM' 'EID_AM' 'Class_AM' 'License_state_AM' 'Specialty_AM'\n 'Firstname_LR' 'Lastname_LR' 'Class_LR' 'Specialty_LR' 'Hotel_LR'\n 'Assignment_LR' 'Phone_LR' 'Email_LR' 'Birthday_LR' 'FirstName_DB'\n 'LastName_DB' 'Class_DB' 'Specialty_DB' 'Phone_DB' 'Email_DB'\n 'Address1_DB' 'Address2_DB' 'City_DB' 'State_DB' 'Zip_DB' 'Gender_DB'\n 'DOB_DB' 'Facility_DB' 'Hotel_DB' 'ActivationDate_DB' 'LicenseName_DB'\n 'LicenseState_DB' 'LicenseNum_DB' 'MobID_DB' 'Status_DB' 'DemobDate_DB'\n 'first_name_fixed' 'last_name_fixed' 'middle_name_fixed' 'phone_fixed'\n 'email_fixed' 'stateofResidency_fixed' 'class_fixed' 'specialty_fixed'\n 'CurrentHotel_fixed' 'CurrentFacility_fixed' 'ActivationDate_fixed'

In [ ]:
file_df = file_df.assign(result=file_df['ProfileID_M'].isin(term_df[['profileid']]))
file_df['Termed'] = file_df['result']
file_df.drop(columns=['result'], inplace=True)

In [ ]:
file_df.to_excel(master_path + "Master Tracking Sheet - Revised.xlsx")